<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TERM3/AI_at_Scale/Assignments/Assignment2/performance_profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import regexp_replace, col, countDistinct, col, sum as spark_sum
from pyspark.sql.functions import count, when, lit, create_map
from pyspark.sql.types import StringType, DoubleType, IntegerType ,StructType,StructField
from pyspark.ml.feature import Imputer, VectorAssembler,StandardScaler
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import time
from pyspark.ml.regression import RandomForestRegressor, DecisionTreeRegressor
from pyspark.ml import PipelineModel

In [ ]:
def run_single_performance_profiling(config):
    """Runs performance profiling for a single Spark configuration."""
    print(f"\nConfiguration: {config}")
    spark = create_spark_session(config["executor_cores"], config["max_cores"], config["executor_memory"])

    if config["parallelism"]:
        spark.conf.set("spark.sql.adaptive.coalescePartitions.parallelismFirst", "true")
        spark.conf.set("spark.default.parallelism", config["parallelism"])

    times = {}
    config_label = f"Cores: {config['executor_cores']}, Max: {config['max_cores']}, Mem: {config['executor_memory']}"
    if config['parallelism']:
        config_label += f", Parallelism: {config['parallelism']}"
    config_results = {'Configuration': config_label}

    try:
        start_time = time.time()
        data_path = "bank-full.csv"
        schema = get_schema()
        data = spark.read.csv(data_path, header=True, schema=schema, sep=';')
        train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)
        times['Data Loading'] = time.time() - start_time
    except Exception as e:
        config_results['Data Loading Error'] = str(e)
        spark.stop()
        return config_results

    try:
        start_time = time.time()
        categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
        numerical_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

        indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_indexed", handleInvalid="keep") for col in categorical_cols]
        encoders = [OneHotEncoder(inputCol=f"{col}_indexed", outputCol=f"{col}_encoded") for col in categorical_cols]

        feature_cols = numerical_cols + [f"{col}_encoded" for col in categorical_cols]

        assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
        label_indexer = StringIndexer(inputCol="y", outputCol="label")

        pipeline = Pipeline(stages=indexers + encoders + [assembler, label_indexer])
        preproc_model = pipeline.fit(train_df)
        train_df_processed = preproc_model.transform(train_df)
        test_df_processed = preproc_model.transform(test_df)
        times['Preprocessing'] = time.time() - start_time
    except Exception as e:
        config_results['Preprocessing Error'] = str(e)
        spark.stop()
        return config_results

    try:
        start_time = time.time()
        rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=10, seed=42)
        model = rf.fit(train_df_processed)
        times['Model Training'] = time.time() - start_time
    except Exception as e:
        config_results['Model Training Error'] = str(e)
        spark.stop()
        return config_results

    try:
        start_time = time.time()
        predictions = model.transform(test_df_processed)
        evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
        accuracy = evaluator.evaluate(predictions)
        times['Model Evaluation'] = time.time() - start_time
        times['Accuracy'] = accuracy
    except Exception as e:
        config_results['Model Evaluation Error'] = str(e)
        spark.stop()
        return config_results

    try:
        total_time = 0.0
        for k, v in times.items():
            if k != 'Accuracy' and isinstance(v, (int, float)):
                total_time += v
        times['Total Time'] = total_time
        config_results.update(times)
    except Exception as e:
        config_results['Total Time Calculation Error'] = str(e)

    spark.stop()
    return config_results

In [ ]:
# Configuration 1
config1 = {"executor_cores": "1", "max_cores": "2", "executor_memory": "1g", "parallelism": None}
results1 = run_single_performance_profiling(config1)
print("\nResults for Configuration 1:")
print(pd.DataFrame([results1]).to_string(index=False))

# Configuration 2
config2 = {"executor_cores": "2", "max_cores": "2", "executor_memory": "1g", "parallelism": None}
results2 = run_single_performance_profiling(config2)
print("\nResults for Configuration 2:")
print(pd.DataFrame([results2]).to_string(index=False))

# Configuration 3
config3 = {"executor_cores": "2", "max_cores": "2", "executor_memory": "1g", "parallelism": "1"}
results3 = run_single_performance_profiling(config3)
print("\nResults for Configuration 3:")
print(pd.DataFrame([results3]).to_string(index=False))

# Configuration 4
config4 = {"executor_cores": "2", "max_cores": "2", "executor_memory": "1g", "parallelism": "2"}
results4 = run_single_performance_profiling(config4)
print("\nResults for Configuration 4:")
print(pd.DataFrame([results4]).to_string(index=False))